# B03. Optimizer
Sources: <br>

Description: This optimizes lineups based on player projections and provided constraints.

### 1. Players

In [14]:
def create_player_file(contestKey, guide, draftGroupId, roto_slate, max_exposure_pitchers, max_exposure_batters):   
    # Read in draftables
    draftables = pd.read_csv(os.path.join(baseball_path, "A01. DraftKings", "2. Draftables", f"Draftables {draftGroupId}.csv"), encoding='iso-8859-1')
    
    # Merge game_id onto draftables
    draftables = pd.merge(draftables, guide[['Game Info', 'game_id']], on='Game Info', how='left')
    
    # Read in players
    player_sims = create_contest_player_sims(guide, contestKey)
    player_sims.rename(columns={'fullName':'Name', 'gamePk':'game_id'}, inplace=True)
    player_sims['game_id'] = player_sims['game_id'].astype('int')
    player_sims['AvgPointsPerGame'] = player_sims.filter(regex='^FP_').mean(axis=1)
    
    # Merge onto draftables
    draftables = pd.merge(draftables, player_sims, on=['Name', 'game_id'], how='inner', suffixes=("", "2"))
    
    # Clean
    draftables['AvgPointsPerGame'] = draftables['AvgPointsPerGame2']
    draftables.drop(columns=['AvgPointsPerGame2'], inplace=True)
    
    # Sort players by average projection
    draftables.sort_values('AvgPointsPerGame', ascending=False, inplace=True)

    draftables['PCT50'] = draftables.filter(like='FP_').quantile(0.5, axis=1)
    draftables['PCT75'] = draftables.filter(like='FP_').quantile(0.75, axis=1)
    draftables['PCT90'] = draftables.filter(like='FP_').quantile(0.9, axis=1)
    draftables['PCT95'] = draftables.filter(like='FP_').quantile(0.95, axis=1)
    draftables['PCT99'] = draftables.filter(like='FP_').quantile(0.99, axis=1)
    draftables['PCT100'] = draftables.filter(like='FP_').quantile(1, axis=1)
                                                                     
    # Add constraints
    draftables['Min Exposure'] = 0
    draftables['Max Exposure'] = np.where(draftables['Roster Position'] == "P", max_exposure_pitchers, max_exposure_batters)
    draftables['Roster Order'] = draftables['batting_order'].copy()
    draftables['Roster Order'] = np.where(draftables['Roster Position'] == "P", 99, draftables['Roster Order']).astype(int)
    draftables['Confirmed Starter'] = np.where(draftables['confirmed'] == 1, 1, 0)
    
    return draftables

### 2. Lineups

In [16]:
def create_lineups(contestKey, min_salary=49000, min_projection=5, major_stack=5, minor_stack=2, excluded_teams=[], min_starters=10, lineups=200):
    ### Load in DraftKings baseball optimizer
    optimizer = get_optimizer(Site.DRAFTKINGS, Sport.BASEBALL)

    ### Load in player sims
    optimizer.load_players_from_csv(os.path.join(baseball_path, "B03. Lineups", "1. Players", f"Players {contestKey}.csv"))

    # for player in optimizer.player_pool._players:
    #     print(player, player.is_confirmed_starter, player.roster_order, player.max_exposure)
    
    ### Settings
    # Set minimum salary
    optimizer.set_min_salary_cap(min_salary)
    # Major Stack
    if major_stack == 5:
        optimizer.add_stack(TeamStack(size=5, spacing=6, for_positions=['C', '1B', '2B', '3B', 'SS', 'OF']))
    elif major_stack == 4:
        optimizer.add_stack(TeamStack(size=4, spacing=5, for_positions=['C', '1B', '2B', '3B', 'SS', 'OF']))
    # Minor Stack
    if minor_stack == 3:
        optimizer.add_stack(TeamStack(size=3, spacing=4, for_positions=['C', '1B', '2B', '3B', 'SS', 'OF']))
    elif minor_stack == 2:
        optimizer.add_stack(TeamStack(size=2, spacing=3, for_positions=['C', '1B', '2B', '3B', 'SS', 'OF']))
    # Position Restrictions
    optimizer.restrict_positions_for_opposing_team(['SP', 'RP'], ['C', 'SS', 'OF', '1B', '2B', '3B']) 
    # Team Exclusions
    optimizer.player_pool.exclude_teams(excluded_teams)
    # Confirmed Starters
    optimizer.set_min_starters(min_starters)
    # Minimum Projection
    optimizer.player_pool.add_filters(
        PlayerFilter(from_value=min_projection),
    )
    # # Ownership
    # optimizer.set_projected_ownership(max_projected_ownership=0.25)
    
    ### Optimizer
    i = 0
    for lineup in optimizer.optimize(lineups, exposure_strategy=AfterEachExposureStrategy):
        if i in [1, 25, 50, 75, 99, 100, 125, 150, 175, 200, 250, 300, 400, 500, 600, 700, 800, 900, 1000]:
                print(i)
        i += 1 
        # if i < 5:
        #     print(lineup)

    # Export lineups to csv
    optimizer.export(os.path.join(baseball_path, "B03. Lineups", "2. Lineups", f"Lineups {contestKey}.csv"))

### 3. Lineups Ranked

In [2]:
def choose_lineups(contestKey, roto_slate, sort_by='Plus3'):
    # Read in players
    player_sims = pd.read_csv(os.path.join(baseball_path, "B03. Lineups", "1. Players", f"Players {contestKey}.csv"))
    
    # Keep relevant variables
    player_sims.drop(columns={"Position", "Name", "ID", "Roster Position", "Salary", "Game Info", "TeamAbbrev", 'playerId', 'draftGroupId', 'game_id', 'Position2', 'imp_l', 'imp_r', "AvgPointsPerGame"}, inplace=True)
    

    # Clean Name + ID variable to remove space (this is for consistency for merging)
    player_sims['Name + ID'] = player_sims['Name + ID'].str.replace(r'\s*\(', '(', regex=True, flags=re.IGNORECASE)
    
    # Determine number of game simulations
    num_sims = sum('FP_' in column_name for column_name in player_sims.columns)

    
    # Read in daily lineups
    lineup_sims = pd.read_csv(os.path.join(baseball_path, "B03. Lineups", "2. Lineups", f"Lineups {contestKey}.csv"))
    
    # Merge stats onto lineups
    lineup_sims = lineup_sims.merge(player_sims, left_on="P", right_on="Name + ID", how='left', validate="m:1")
    lineup_sims = lineup_sims.merge(player_sims, left_on="P.1", right_on="Name + ID", how='left', validate="m:1", suffixes=(None, "_P.1"))
    lineup_sims = lineup_sims.merge(player_sims, left_on="C", right_on="Name + ID", how='left', validate="m:1", suffixes=(None, "_C"))
    lineup_sims = lineup_sims.merge(player_sims, left_on="1B", right_on="Name + ID", how='left', validate="m:1", suffixes=(None, "_1B"))
    lineup_sims = lineup_sims.merge(player_sims, left_on="2B", right_on="Name + ID", how='left', validate="m:1", suffixes=(None, "_2B"))
    lineup_sims = lineup_sims.merge(player_sims, left_on="3B", right_on="Name + ID", how='left', validate="m:1", suffixes=(None, "_3B"))
    lineup_sims = lineup_sims.merge(player_sims, left_on="SS", right_on="Name + ID", how='left', validate="m:1", suffixes=(None, "_SS"))
    lineup_sims = lineup_sims.merge(player_sims, left_on="OF", right_on="Name + ID", how='left', validate="m:1", suffixes=(None, "_OF"))
    lineup_sims = lineup_sims.merge(player_sims, left_on="OF.1", right_on="Name + ID", how='left', validate="m:1", suffixes=(None, "_OF.1"))
    lineup_sims = lineup_sims.merge(player_sims, left_on="OF.2", right_on="Name + ID", how='left', validate="m:1", suffixes=(None, "_OF.2"))

    
    # Add up player performances
    i=0
    # Where i is the number of simulations
    while i < num_sims:
        sim = f"FP_{i}"
        P1 = sim
        P2 = sim + "_P.1"
        C = sim + "_C"
        B1 = sim + "_1B"
        B2 = sim + "_2B"
        B3 = sim + "_3B"
        SS = sim + "_SS"
        OF1 = sim + "_OF"
        OF2 = sim + "_OF.1"
        OF3 = sim + "_OF.2"

        game = f"Sim {i}"

        lineup_sims[game] = lineup_sims[P1] + lineup_sims[P2] + lineup_sims[C] + lineup_sims[B1] + lineup_sims[B2] + lineup_sims[B3] + lineup_sims[SS] + lineup_sims[OF1] + lineup_sims[OF2] + lineup_sims[OF3]

        i+=1

    # Delete excess variables
    lineup_sims.rename(columns={'FPPG':'AvgPointsPerGame'}, inplace=True)
    lineup_sims = lineup_sims.loc[:, ~lineup_sims.columns.str.contains('FP', case=False)]
    lineup_sims = lineup_sims.loc[:, ~lineup_sims.columns.str.contains('Name', case=False)]
    lineup_sims = lineup_sims.loc[:, ~lineup_sims.columns.str.contains('Order', case=False)]
    lineup_sims = lineup_sims.loc[:, ~lineup_sims.columns.str.contains('Exposure', case=False)]

    print(lineup_sims.shape)
    
    try:
        # Read in ownership (RotoWire)
        roto_df = pd.read_csv(os.path.join(baseball_path, "A07. Projections", "2. RotoWire", "2. Projections", f"RotoWire Projections {roto_slate}.csv"))

        # Create name
        roto_df['Name'] = roto_df['firstName'] + " " + roto_df['lastName']

        # Clean name
        roto_df = name_clean(roto_df, 'Name')

        # Only keep rostership (and name to merge)
        roto_df = roto_df[['Name', 'rostership']]
        # Keep only one of each name - can probably find a better solution later
        roto_df.drop_duplicates(subset=['Name'], keep='first', inplace=True)
        
        
        # Calculate total rostership - sometimes it'll be weirdly low, which is an error by RotoWire
        rostership_sum = roto_df[['rostership']].sum(axis=0)
        # Consider breaking if rostership_sum is too low

        # Loop over position
        for pos in ['P', 'P.1', 'C', '1B', '2B', '3B', 'SS', 'OF', 'OF.1', 'OF.2']:
            # Remove (DraftKingsID)
            lineup_sims['Name'] = lineup_sims[pos].str.split(r"\(", expand=True)[0]

            # Merge to get ownership
            lineup_sims = lineup_sims.merge(roto_df, on='Name', how='left')

            # Rename ownership variable to be position-specific
            lineup_sims.rename(columns={'rostership':f'{pos}_rostership'}, inplace=True)

            # May be missing. This is common for Luis Garcia if we keep the wrong one.
            lineup_sims.fillna(5, inplace=True)

        ### Calculate summary statistics
        column_list = [col for col in lineup_sims if col.startswith("Sim")]

        ### Points
        lineup_sims['AVG'] = lineup_sims[column_list].mean(axis=1)
        lineup_sims['P50'] = lineup_sims[column_list].median(axis=1)
        lineup_sims['P75'] = lineup_sims[column_list].quantile(.75, axis=1)
        lineup_sims['P90'] = lineup_sims[column_list].quantile(.90, axis=1)
        lineup_sims['P95'] = lineup_sims[column_list].quantile(.95, axis=1)
        lineup_sims['P99'] = lineup_sims[column_list].quantile(.99, axis=1)
        lineup_sims['P100'] = lineup_sims[column_list].max(axis=1)

        # Tail fatness
        lineup_sims['Tail'] = 0 
        for column in column_list:
            for i in range(len(lineup_sims)):
                if lineup_sims[column][i] >= lineup_sims['P95'][i]:
                    lineup_sims['Tail'][i] = lineup_sims['Tail'][i] + lineup_sims[column][i]

        lineup_sims['Sim STD'] = lineup_sims[lineup_sims.columns[lineup_sims.columns.str.startswith('Sim')]].std(axis=1)

        # Standard deviations from mean 
        lineup_sims['Plus2'] = lineup_sims['AvgPointsPerGame'] + 2 * lineup_sims['Sim STD']
        lineup_sims['Plus3'] = lineup_sims['AvgPointsPerGame'] + 3 * lineup_sims['Sim STD']
    except:
        print("No ownership projections for this slate.")
       
    
    try:
        ### Ownership
        # Total
        lineup_sims['rostership'] = lineup_sims.filter(like='_rostership').sum(axis=1)
        # Pitcher ownership 
        lineup_sims['pitcher rostership'] = lineup_sims[['P_rostership', 'P.1_rostership']].sum(axis=1)
        # Batter ownership 
        lineup_sims['batter rostership'] = lineup_sims[['C_rostership', '1B_rostership', '2B_rostership', '3B_rostership', 'SS_rostership', 'OF_rostership', 'OF.1_rostership', 'OF.2_rostership']].sum(axis=1)
    except:
        pass

    
    # Identify pareto optimal lineups
    lineup_sims['pareto'] = paretoset(lineup_sims[["AvgPointsPerGame", "Sim STD", 'batter rostership']], sense=['max', 'max', 'min']).astype('int')
    
    # Put them on top
    lineup_sims = lineup_sims.sort_values(by=['pareto', sort_by], ascending=[False, False])
    lineup_sims = lineup_sims.reset_index(drop=True)

    
    # # Sort (descending - Note that DK will read this the wrong way)
    # lineup_sims.sort_values(by=sort_by, ascending=False, inplace=True)
    
    return lineup_sims

### 4. Uploads

In [22]:
def create_upload_file(contestKey, sort_by='Plus3'):
    # Read in lineup sims
    lineup_ranked = pd.read_csv(os.path.join(baseball_path, "B03. Lineups", "3. Lineups Ranked", f"Lineups Ranked {contestKey}.csv"))
    # # Sort (ascending because DK will put the bottom lineups at the top)
    # lineup_ranked.sort_values(by=sort_by, ascending=True, inplace=True)
    # Keep just the players
    lineup_ranked = lineup_ranked[['P', 'P.1', 'C', '1B', '2B', '3B', 'SS', 'OF', 'OF.1', 'OF.2']]
    
    # Rename variables to appease DK's upload
    lineup_ranked.rename(columns={'P.1':'P', 'OF.1':'OF', 'OF.2':'OF'}, inplace=True)
    
    return lineup_ranked

### 5. Entries

In [1]:
def create_entry_file(draftGroupId, contestKey):
    # Download entry file for draftGroupId
    url = f"https://www.draftkings.com/bulkentryedit/getentriescsv?draftGroupId={draftGroupId}"

    javascript_code = f"window.open('{url}', '_blank');"
    display(Javascript(javascript_code))
    
    time.sleep(5)
    
    # Get the list of files in the downloads folder
    files = os.listdir(download_path)

    # Get the most recently modified file (entry sheet)
    most_recent_file = max(files, key=lambda x: os.path.getctime(os.path.join(download_path, x)))
    most_recent_file_path = os.path.join(download_path, most_recent_file)
    df = pd.read_csv(most_recent_file_path, usecols=['Entry ID','Contest Name','Contest ID','Entry Fee'])
    df.dropna(inplace=True)
    
    # Read in Upload file
    lineup_sims = pd.read_csv(os.path.join(baseball_path, "B03. Lineups", "4. Uploads", f"Upload {contestKey}.csv"), encoding='iso-8859-1')
    # # Reverse order (upload file places top lineups at the bottom)
    # lineup_sims = lineup_sims[::-1]
    # Keep just the players
    lineup_sims = lineup_sims[['P', 'P.1', 'C', '1B', '2B', '3B', 'SS', 'OF', 'OF.1', 'OF.2']]
    # Rename variables to appease DK's upload
    lineup_sims.rename(columns={'P.1':'P', 'OF.1':'OF', 'OF.2':'OF'}, inplace=True)
    lineup_sims.reset_index(inplace=True, drop=True)

    # Merge entry sheet with lineups
    entry_df = df.merge(lineup_sims, how='inner', left_index=True, right_index=True)

    # Convert to numeric
    entry_df['Entry ID'] = entry_df['Entry ID'].astype('int64')

    return entry_df

### Run

In [ ]:
def create_contest_lineups(contestKey, sort_by, min_salary, min_projection, major_stack, minor_stack, max_exposure_batters, max_exposure_pitchers, excluded_teams, min_starters, lineups, historic):
    # Read in Contest Guide
    guide = pd.read_csv(os.path.join(baseball_path, "A09. Contest Guides", f"Contest Guide {contestKey}.csv"))

    # Identify draftGroupId
    draftGroupId = guide['draftGroupId'][0]

    # Identify date
    date = guide['date'][0]

    # Identify RotoWire slate
    roto_slate = guide['roto_slate'][0]
    
    # 1. Players
    # This creates player files to be used as inputs in optimizer
    draftables_with_sims = create_player_file(contestKey, guide, draftGroupId, date, roto_slate)    
    draftables_with_sims.to_csv(os.path.join(baseball_path, "B03. Lineups", "1. Players", f"Players {contestKey}.csv"), index=False, encoding='iso-8859-1')
    
    # 2. Lineups
    # This creates optimal lineups
    create_lineups(contestKey, min_salary, min_projection, major_stack, minor_stack, excluded_teams, min_starters, lineups)
    
    # 3. Lineups Ranked
    # This adds stats based on score distributions to assess which lineups to choose
    lineups_ranked = choose_lineups(contestKey, roto_slate, sort_by)
    lineups_ranked.to_csv(os.path.join(baseball_path, "B03. Lineups", "3. Lineups Ranked", f"Lineups Ranked {contestKey}.csv"), index=False)
    
    # 4. Uploads
    # This creates a file to upload lineups to DraftKings in the proper order
    if historic == False:
        # Create upload file
        upload = create_upload_file(contestKey, sort_by)
        upload.to_csv(os.path.join(baseball_path, "B03. Lineups", "4. Uploads", f"Upload {contestKey}.csv"), index=False)

    # 5. Entries
    # This creates a file to upload entry-specific lineups
    if historic == False:
        entry = create_entry_file(draftGroupId, contestKey)
        entry.to_csv(os.path.join(baseball_path, "B03. Lineups", "5. Entries", f"Entries {draftGroupId}.csv"), index=False, encoding='iso-8859-1')

In [ ]:
# This returns contestKeys that do not work
def create_contest_lineups2(contestKey, sort_by, min_salary, min_projection, major_stack, minor_stack, max_exposure_batters, max_exposure_pitchers, excluded_teams, min_starters, lineups, historic):
    try:
        return create_contest_lineups(contestKey, sort_by, min_salary, min_projection, major_stack, minor_stack, max_exposure_batters, max_exposure_pitchers, excluded_teams, min_starters, lineups, historic)
    except Exception as e:
        print(f"Error processing contestKey: {contestKey}. Exception: {e}")
        return contestKey

### Email

In [26]:
def email_upload_file(draftGroupId, contestKey):    
    message = f"""\
    draftGroupId: {draftGroupId}
    contestKey: {contestKey}

    Entries: https://www.draftkings.com/entry/upload
    Uploads: https://www.draftkings.com/lineup/upload
    """

    sender_email = 'jamesgiles1993@gmail.com'
    receiver_email = 'jamesgiles1993@gmail.com'
    smtp_server = 'smtp.gmail.com'
    port = 465
    password = 'uepgnvemqxttdxbq'

    # Create a multipart message object
    msg = MIMEMultipart()
    msg['Subject'] = f'Lineups: {contestKey}' 
    msg['From'] = sender_email
    msg['To'] = receiver_email

    # Attach the message to the email
    msg.attach(MIMEText(message, 'plain'))

    # Add Entry and Upload files as attachments
    entry_path = os.path.join(baseball_path, "B03. Lineups", "5. Entries", f"Entries {draftGroupId}.csv")
    upload_path = os.path.join(baseball_path, "B03. Lineups", "4. Uploads", f"Upload {contestKey}.csv")

    with open(entry_path, 'rb') as attachment:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename="{entry_path}"')
        msg.attach(part)
    
    with open(upload_path, 'rb') as attachment:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename="{upload_path}"')
        msg.attach(part)

    # Create a secure SSL context
    context = ssl.create_default_context()

    # Send the email
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

### Upload

In [ ]:
def upload_entries(draftGroupId):
    # Open entry page
    webbrowser.open(f"https://www.draftkings.com/entry/upload")
    time.sleep(7)

    # Search for "UPLOAD CSV" and get its position
    upload_csv_button = pyautogui.locateOnScreen(r"C:\Users\james\Documents\MLB\UPLOAD CSV2.png", confidence=0.8)
    
    # Check if the button is found
    if upload_csv_button is not None:
        # If found, click on it
        pyautogui.click(upload_csv_button)
    else:
        print("Button not found.")
    
    # Access directory bar
    pyautogui.hotkey('alt', 'd')
    time.sleep(1)

    # Type filepath
    pyautogui.typewrite(rf"C:\Users\james\Documents\MLB\Database\B03. Lineups\5. Entries\Entries {draftGroupId}.csv")
    time.sleep(1)
    pyautogui.press('enter')

### Run One

In [ ]:
# # Create one day's lineups
# create_lineups(200, 1000, todaysdate)
# choose_lineups(1000, todaysdate)
    
# create_upload_file(todaysdate)
# print("https://www.draftkings.com/lineup/upload")
# email_upload_file(todaysdate)

### Run All

In [ ]:
# # Create a second function that won't break
# def create_lineups2(lineups=200, sims=1000, date=todaysdate):
#     try:
#         create_lineups(lineups, sims, date)
#         choose_lineups(sims, date)
#     except:
#         pass
    
# # Identify all days for which there are player sims
# days = []
# for filename in os.listdir(r"C:\Users\james\Documents\MLB\Data\A8. Sims - 1. Players"): 
#     # 2023 
#     if filename.endswith(".csv") and filename.startswith("Player_Sims_2022"):
#         # Pull out date
#         date = filename[12:20]
#         days.append(date)


# # Run all in parallel
# Parallel(n_jobs=-2, verbose=5)(delayed(create_lineups2)(200, 1000, day) for day in days)

In [ ]:
# print("Code was last run on: {} at {}.".format(datetime.date.today(), datetime.datetime.now().strftime("%H:%M:%S")))